In [5]:
!pip install tensorflow==2.16.2
!pip install keras-tuner==1.4.7
!pip install numpy<2.0.0
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.7/590.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: ml-dtypes
    Fo

In [15]:
# 1. Setting up Keras Tuner
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

(x_train,y_train),(x_val,y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

# 계속해서 validatioin_acc 가 0.05또는 거의 0에 가까운 정확도가 나와서
# epoch 수도 늘려보고, model 레이어, 활성화 함수도 바꾸어 봤지만 계속 똑같은 결과가 나와서 짜증나던중
# log 값을 보니 학습도중에 val_accuracy 값은 최대 0.95까지 도달하는데 model.evaluate 에서는 0.005 수준이
# 나와서 성능은 좋아졌다고 하는데 테스트에서 엉망이라 뭐가 잘못된걸까 코드를 보던중
# x_train, x_val 값에 대한 데이터 전처리가 이루어졌어야 했는데
# y_val 값을 정규화 하고 있었다!!!!!!!!! - y값은 정수값으로 0-9 였어야 했는데 의미없게 255.0 으로 나누어놨던것....
# 작은 코드라도 제대로 봐야한다는걸 다시 한번 느낌...

# 실수 한 부분 고정!
# (x_train,y_train),(x_val,y_val) = mnist.load_data()
#x_val, y_val  = x_val/255.0 , y_val/255.0

print(f'Shape of x_train: {x_train.shape}')
print(f'Shape of x_test: {x_val.shape}')

# 2. build model

def build_model(hp):
    model = Sequential([
              Flatten(input_shape=(28,28)),
              Dense(units=hp.Int("units", min_value=64, max_value=512, step=64), activation='relu'),
              Dropout(0.3),
              Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-3, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

#3.Configuring the hyperparameter search objective

tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    directory="my_dir",
    project_name="mnist_to_keras_tuner"
)

tuner.search(x_train, y_train, epochs=15, validation_data=(x_val, y_val))  # 에포크 증가


# 4. Running the hyperparameter search

tuner.results_summary()

# 5. Analyzing and using the best hyperparameter
best_hy = tuner.get_best_hyperparameters(num_trials = 1)[0]
# (num_trials = 1) 가장 성능이 좋았던 "1"개의 파라미터 조합을 가져옴
# [0] 그중에서 첫번쨰 값을 불러오기
print(f"""

The optimal number of units in the first dense layer is {best_hy['units']}.

The optimal learning rate for the optimizer is {best_hy['learning_rate']}.

""")

# Rebuild and train the model with best hyperparameters
model = tuner.hypermodel.build(best_hy)
history = model.fit(x_train, y_train, epochs = 10, validation_split= 0.2)

#evaluate the model on the validation set
val_loss, val_acc = model.evaluate(x_val, y_val)
print(f"Validation accuracy: {val_acc}")




Shape of x_train: (60000, 28, 28)
Shape of x_test: (10000, 28, 28)
Reloading Tuner from my_dir/mnist_to_keras_tuner/tuner0.json
Results summary
Results in my_dir/mnist_to_keras_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
units: 256
learning_rate: 0.0001064456532939776
Score: 0.09314999729394913

Trial 02 summary
Hyperparameters:
units: 512
learning_rate: 0.00042541799335877174
Score: 0.07719999924302101

Trial 07 summary
Hyperparameters:
units: 448
learning_rate: 0.0004080644937450095
Score: 0.06815000250935555

Trial 00 summary
Hyperparameters:
units: 256
learning_rate: 0.00028228527126761055
Score: 0.047799999825656414

Trial 03 summary
Hyperparameters:
units: 320
learning_rate: 0.0005146912109442893
Score: 0.022450000513345003

Trial 06 summary
Hyperparameters:
units: 64
learning_rate: 0.00011042062841724865
Score: 0.016500000085216016

Trial 04 summary
Hyperparameters:
units: 128
learning_rate: 0.000131909123021043

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7006 - loss: 1.0598 - val_accuracy: 0.9197 - val_loss: 0.3043
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9072 - loss: 0.3323 - val_accuracy: 0.9352 - val_loss: 0.2380
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9229 - loss: 0.2708 - val_accuracy: 0.9473 - val_loss: 0.1965
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9381 - loss: 0.2212 - val_accuracy: 0.9527 - val_loss: 0.1723
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9480 - loss: 0.1899 - val_accuracy: 0.9586 - val_loss: 0.1549
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9509 - loss: 0.1733 - val_accuracy: 0.9605 - val_loss: 0.1414
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9571 - loss: 0.1522 - val_accuracy: 0.9641 - val_loss: 0.1307
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9611 - loss: 0.134